# 영업 성공 여부 분류 경진대회

## 1. 데이터 확인

### 필수 라이브러리

In [18]:
import pandas as pd
import numpy as np
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

import matplotlib.pyplot as plt

#### 데이터 셋 읽어오기

In [19]:
df_train = pd.read_csv("train1.csv") # 학습용 데이터
df_test = pd.read_csv("submission1.csv") # 테스트 데이터(제출파일의 데이터)

In [20]:
df_train.head() # 학습용 데이터 살펴보기

,bant_submit,customer_country,business_unit,com_reg_ver_win_rate,customer_idx,customer_type,enterprise,historical_existing_cnt,id_strategic_ver,it_strategic_ver,...,response_corporate,expected_timeline,ver_cus,ver_pro,ver_win_rate_x,ver_win_ratio_per_bu,business_area,business_subarea,lead_owner,is_converted
0,1.0,/Quezon City/Philippines,AS,0.066667,32160,End-Customer,Enterprise,NaN,NaN,NaN,...,LGEPH,less than 3 months,1,0,0.003079,0.026846,corporate / office,Engineering,0,True
1,1.0,/PH-00/Philippines,AS,0.066667,23122,End-Customer,Enterprise,12.0,NaN,NaN,...,LGEPH,less than 3 months,1,0,0.003079,0.026846,corporate / office,Advertising,1,True
2,1.0,/Kolkata /India,AS,0.088889,1755,End-Customer,Enterprise,144.0,NaN,NaN,...,LGEIL,less than 3 months,1,0,0.003079,0.026846,corporate / office,Construction,2,True
3,1.0,/Bhubaneswar/India,AS,0.088889,4919,End-Customer,Enterprise,NaN,NaN,NaN,...,LGEIL,less than 3 months,1,0,0.003079,0.026846,corporate / office,IT/Software,3,True
4,1.0,/Hyderabad/India,AS,0.088889,17126,Specifier/ Influencer,Enterprise,NaN,NaN,NaN,...,LGEIL,less than 3 months,0,0,0.003079,0.026846,corporate / office,NaN,4,True


In [21]:
df_train.shape

(59299, 29)

In [22]:
df_test.shape

(5271, 30)

In [23]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59299 entries, 0 to 59298
Data columns (total 29 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   bant_submit              59299 non-null  float64
 1   customer_country         58317 non-null  object 
 2   business_unit            59299 non-null  object 
 3   com_reg_ver_win_rate     14568 non-null  float64
 4   customer_idx             59299 non-null  int64  
 5   customer_type            15338 non-null  object 
 6   enterprise               59299 non-null  object 
 7   historical_existing_cnt  13756 non-null  float64
 8   id_strategic_ver         3444 non-null   float64
 9   it_strategic_ver         1121 non-null   float64
 10  idit_strategic_ver       4565 non-null   float64
 11  customer_job             40566 non-null  object 
 12  lead_desc_length         59299 non-null  int64  
 13  inquiry_type             58358 non-null  object 
 14  product_category      

오늘은 bant_submit이 무슨 영향을 끼치는지 알아보자.

plot을 그리기 전에 결측값이나 NaN값이 숨어있는지 확인.

# 2. 데이터 전처리

### 레이블 인코딩

In [24]:
def label_encoding(series: pd.Series) -> pd.Series:
    """범주형 데이터를 시리즈 형태로 받아 숫자형 데이터로 변환합니다."""

    my_dict = {}

    # 모든 요소를 문자열로 변환
    series = series.astype(str)

    for idx, value in enumerate(sorted(series.unique())):
        my_dict[value] = idx
    series = series.map(my_dict)

    return series

In [25]:
# 레이블 인코딩할 칼럼들
label_columns = [
    "customer_country",
    "business_subarea",
    "business_area",
    "business_unit",
    "customer_type",
    "enterprise",
    "customer_job",
    "inquiry_type",
    "product_category",
    "product_subcategory",
    "product_modelname",
    "customer_country.1",
    "customer_position",
    "response_corporate",
    "expected_timeline",
]

df_all = pd.concat([df_train[label_columns], df_test[label_columns]])

for col in label_columns:
    df_all[col] = label_encoding(df_all[col])

In [26]:
for col in label_columns:  
    df_train[col] = df_all.iloc[: len(df_train)][col]
    df_test[col] = df_all.iloc[len(df_train) :][col]

In [27]:
df_train.head()

,bant_submit,customer_country,business_unit,com_reg_ver_win_rate,customer_idx,customer_type,enterprise,historical_existing_cnt,id_strategic_ver,it_strategic_ver,...,response_corporate,expected_timeline,ver_cus,ver_pro,ver_win_rate_x,ver_win_ratio_per_bu,business_area,business_subarea,lead_owner,is_converted
0,1.0,9070,0,0.066667,32160,10,0,NaN,NaN,NaN,...,33,246,1,0,0.003079,0.026846,0,28,0,True
1,1.0,8406,0,0.066667,23122,10,0,12.0,NaN,NaN,...,33,246,1,0,0.003079,0.026846,0,0,1,True
2,1.0,6535,0,0.088889,1755,10,0,144.0,NaN,NaN,...,21,246,1,0,0.003079,0.026846,0,17,2,True
3,1.0,3388,0,0.088889,4919,10,0,NaN,NaN,NaN,...,21,246,1,0,0.003079,0.026846,0,44,3,True
4,1.0,5799,0,0.088889,17126,29,0,NaN,NaN,NaN,...,21,246,0,0,0.003079,0.026846,0,86,4,True


In [28]:
df_test.head()

,id,bant_submit,customer_country,business_unit,com_reg_ver_win_rate,customer_idx,customer_type,enterprise,historical_existing_cnt,id_strategic_ver,...,response_corporate,expected_timeline,ver_cus,ver_pro,ver_win_rate_x,ver_win_ratio_per_bu,business_area,business_subarea,lead_owner,is_converted
0,19844,0.00,11,2,0.073248,47466,9,0,53.0,NaN,...,43,271,1,0,0.001183,0.049840,10,25,278,True
1,9738,0.25,15156,3,NaN,5405,9,1,NaN,NaN,...,50,271,0,0,0.000013,NaN,12,62,437,True
2,8491,1.00,53,2,NaN,13597,29,1,NaN,NaN,...,18,246,0,0,0.000060,0.131148,4,37,874,False
3,19895,0.50,893,2,0.118644,17204,33,0,NaN,NaN,...,50,267,0,0,0.001183,0.049840,10,86,194,False
4,10465,1.00,1402,2,0.074949,2329,9,0,2.0,1.0,...,43,246,1,1,0.003079,0.064566,0,28,167,False


In [16]:
# CSV 파일에 저장
df_train.to_csv("encoded_data.csv")

각 레이블마다 NaN값이 몇개인지 알아보자.

In [46]:
# train data set 레이블

label_columns = [
    "bant_submit",
    "customer_country",
    "business_unit",
    "com_reg_ver_win_rate",
    "business_area",
    "customer_idx",
    "customer_type",
    "enterprise",
    "historical_existing_cnt",
    "id_strategic_ver",
    "it_strategic_ver",
    "idit_strategic_ver",
    "customer_job",
    "lead_desc_length",
    "inquiry_type",
    "product_category",
    "product_subcategory",
    "product_modelname",
    "customer_country.1",
    "customer_position",
    "response_corporate",
    "expected_timeline",
    "ver_cus",
    "ver_pro",
    "ver_win_rate_x",
    "ver_win_ratio_per_bu",
    "business_area",
    "business_subarea",
    "lead_owner",
    "is_converted"
]

for column_name in label_columns:
    nan_count = df_train[column_name].isnull().sum()
    print(column_name, "열의 NaN 값 개수:", nan_count)

bant_submit 열의 NaN 값 개수: 0
customer_country 열의 NaN 값 개수: 0
business_unit 열의 NaN 값 개수: 0
com_reg_ver_win_rate 열의 NaN 값 개수: 44731
business_area 열의 NaN 값 개수: 0
customer_idx 열의 NaN 값 개수: 0
customer_type 열의 NaN 값 개수: 0
enterprise 열의 NaN 값 개수: 0
historical_existing_cnt 열의 NaN 값 개수: 45543
id_strategic_ver 열의 NaN 값 개수: 55855
it_strategic_ver 열의 NaN 값 개수: 58178
idit_strategic_ver 열의 NaN 값 개수: 54734
customer_job 열의 NaN 값 개수: 0
lead_desc_length 열의 NaN 값 개수: 0
inquiry_type 열의 NaN 값 개수: 0
product_category 열의 NaN 값 개수: 0
product_subcategory 열의 NaN 값 개수: 0
product_modelname 열의 NaN 값 개수: 0
customer_country.1 열의 NaN 값 개수: 0
customer_position 열의 NaN 값 개수: 0
response_corporate 열의 NaN 값 개수: 0
expected_timeline 열의 NaN 값 개수: 0
ver_cus 열의 NaN 값 개수: 0
ver_pro 열의 NaN 값 개수: 0
ver_win_rate_x 열의 NaN 값 개수: 40882
ver_win_ratio_per_bu 열의 NaN 값 개수: 43995
business_area 열의 NaN 값 개수: 0
business_subarea 열의 NaN 값 개수: 0
lead_owner 열의 NaN 값 개수: 0
is_converted 열의 NaN 값 개수: 0


In [51]:
df_dropna_train = df_train.dropna(axis = 1, thresh = 55000)

In [54]:
df_dropna_train.head()

,bant_submit,customer_country,business_unit,customer_idx,customer_type,enterprise,customer_job,lead_desc_length,inquiry_type,product_category,...,customer_country.1,customer_position,response_corporate,expected_timeline,ver_cus,ver_pro,business_area,business_subarea,lead_owner,is_converted
0,1.0,9070,0,32160,10,0,420,62,32,180,...,9070,47,33,246,1,0,0,28,0,True
1,1.0,8406,0,23122,10,0,303,96,32,180,...,8406,21,33,246,1,0,0,0,1,True
2,1.0,6535,0,1755,10,0,160,56,27,238,...,6535,80,21,246,1,0,0,17,2,True
3,1.0,3388,0,4919,10,0,166,44,32,318,...,3388,21,21,246,1,0,0,44,3,True
4,1.0,5799,0,17126,29,0,84,97,32,180,...,5799,80,21,246,0,0,0,86,4,True


시각화좀 하고싶다..

In [67]:
def bar_chart(feature):
    # 각 컬럼에서 not이탈자 수 count
    converted = df_dropna_train[df_dropna_train["is_converted"] == 1][feature].value_counts()

    # 각 컬럼에서 이탈자 수 count
    not_converted = df_dropna_train[df_dropna_train["is_converted"] == 0][feature].value_counts()

    # 이탈자와 not이탈자를 하나의 dataframe으로 묶는다.
    df = pd.DataFrame([converted, not_converted])

    # 묶은 dataframe의 인덱스명(행, 이름)을 지정한다.
    df.index = ["converted", "not_converted"]

    # 그래프를 그린다.
    df.plot(kind = "bar",stacked=True, figsize = (8, 8))

In [68]:
bar_chart("customer_country")

ValueError: Image size of 693x322685 pixels is too large. It must be less than 2^16 in each direction.

<Figure size 800x800 with 1 Axes>